In [25]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

X=pd.read_csv('../data/processed/X.csv')
Y=pd.read_csv('../data/processed/Y.csv')

In [26]:
# h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.1-internal" 2018-10-16; OpenJDK Runtime Environment (build 11.0.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5x4orxnb
  JVM stdout: /tmp/tmp5x4orxnb/h2o_carpiero_started_from_python.out
  JVM stderr: /tmp/tmp5x4orxnb/h2o_carpiero_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_carpiero_9fre1l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.686 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [23]:
X_Y=pd.concat([X,Y], axis=1)


In [29]:
x= X.columns.to_list()
y = "price"

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_int',
 'color_int',
 'clarity_int',
 'raw_density']

In [ ]:
train_X_Y=h2o.H2OFrame(X_Y)
aml = H2OAutoML(max_models=50, max_runtime_secs= 300, seed= 32,sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_X_Y)


In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
model= aml.leader
scores = cross_val_score(model, X, Y, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
print(f' RMSE_CROSS = {np.mean(-scores)}\n\n')

In [ ]:
h2o.cluster().shutdown()